# About this Notebook

The purpose of this notebook is threefold: <br> 
    1) Explore how to use the NASDAQ Data Link API <br>
    2) Retrieve stock information packaged in the familiar JSON format <br>
    3) Assess the data using only an economy of packages (i.e., no Pandas)

Our first step is to obtain an API key for the NASDAQ site and set it as an environment variable

In [2]:
import os
import json
import requests
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv(key='NASDAQ_API_KEY')

Now that we have gained access to the API, we'll query the site for data from the Frankfurt Stock Exchange (FSE) for data on Carl Zeiss Meditec (AFX_X). Our query specifies not only the data we want, but also the format in which we would like to receive the data.  In this case, it's the JSON file format. 

In [4]:
# specify URL API
url = f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key={API_KEY}"

# send a GET request to the API
r = requests.get(url)

# Convert the JSON file to a Python dictionary and view
response = r.json()

We previewed the response and saw that our first key was "dataset_data."  The output remains a bit overwhelming.  Let's continue by identifying the keys with which we have to work.  We'll only examine the keys whose associated value is something other than None.

In [8]:
# take the first key available
initial_key = 'dataset_data'

# collect the remaining keys
def get_keys(json_file):
    keys = []
    for k in json_file.keys():
        if k not in keys:
            keys.append(k)
    return keys

# call our function and view its output
resp_keys = get_keys(response[initial_key])
print(resp_keys)

['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order']


We have a number of keys in our file.  Below, we discover what values are associated with each key.

In [9]:
# cycle through keys to view output
for entry in resp_keys:
    if response[initial_key][entry]:
        print(str(entry) + ":" + str(response[initial_key][entry][0]))

column_names:Date
start_date:2
end_date:2
frequency:d
data:['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]


The output above indicates our file follows the expected pattern for a JSON file: a mixture of keys associated with values and keys associated with nested data structures containing values.  

It looks like `column_names` are stored separately from the data they describe. We'll take the column names and pair them with a list of data points -- accessed via the aptly-named `data` key -- to see what type of data we have for each trading day in a user-friendly format.

In [10]:
columns = response['dataset_data']['column_names']
values = response['dataset_data']['data'][0]
zipped_file = zip(columns, values)
for k, v in zipped_file:
    print(k + " : " + str(v))

Date : 2020-12-01
Open : 112.2
High : 112.2
Low : 111.5
Close : 112.0
Change : None
Traded Volume : 51.0
Turnover : 5703.0
Last Price of the Day : None
Daily Traded Units : None
Daily Turnover : None


We have struck gold!  Based on the above output, we can discern the index values for each data point (e.g., date = 0, open = 1, etc).  This makes accessing individual data points easy. We can also anticipate which values are most likely to be `None`. 

Having a firm grasp of the structure and navigation of this JSON file better positions us to answer some questions about the data.

### Tasks 1 & 2 
##### Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).  Convert the returned JSON object into a Python dictionary.



These tasks are most easily accomplished using a second API call with a slight adjustement to our initial request line to accomodate the dates.

In [11]:
# specify URL API
url_2017 = f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?&start_date=2017-01-01&end_date=2017-12-31&api_key={API_KEY}"

# send a GET request to the API
r = requests.get(url_2017)

# Convert the JSON file to a Python dictionary and view
response_2017 = r.json()
print(type(response_2017))

<class 'dict'>


### Task 3
##### Calculate what the highest and lowest opening prices were for the stock in this period.

Our exploration of the JSON file pays off! We know what the initial key is and how the data are packaged. Furthermore, we know that the opening prices for each day are positionally aligned at index 1. 

In [12]:
# store all open prices
opening_prices = []

# loop acress all lists in the data key (using our knowledge and initial key from above)
for i in response_2017[initial_key]['data']:
    if i[1] is not None:
        opening_prices.append(i[1])

# find and print answers
max_open = max(opening_prices)
min_open = min(opening_prices)
print("Max Opening Price: " + str(max_open) + "\n" + "Minimum Opening Price: " + str(min_open))

Max Opening Price: 53.11
Minimum Opening Price: 34.0


### Task 4
##### What was the largest change in any one day (based on High and Low price)?

The question is a bit ambiguous, as it does not specify if we are to calculate percentage change or dollar change. Absent instructions to the contrary, we'll return the dollar change.

In [13]:
highs, lows = [], []

# get high and low prices for each day
for i in response_2017[initial_key]['data']:
    if i[2] is not None and i[3] is not None:
        highs.append(i[2])
        lows.append(i[3])

zipped_high_low = zip(highs, lows)

# set the change variable
max_change_high_low = float("-inf")

# calculate the max change for a single day
for high, low in zipped_high_low:
    # the check for "None" is included once again as a fail safe measure
    if high is not None and low is not None:
        change_high_low = high - low
        max_change_high_low = max(change_high_low, max_change_high_low)

print("The largest change in a single day based on high and low prices: " + "$" + str(round(max_change_high_low, 4)) + ".")

The largest change in a single day based on high and low prices: $2.81.


### Task 5
##### What was the largest change between any two days (based on Closing Price)?

The questions suffer from the same ambiguity as above, so we'll take the same approach as we did in the preceeding cell. 

In [14]:
closing_prices = []

# get closing prices
for i in response_2017[initial_key]['data']:
    if i[4] is not None:
        closing_prices.append(i[4])

# set the change variable
max_change_closing = float("-inf")

# calculate the max change for a single day
for j in range(len(closing_prices)-1):
    if closing_prices[j] and closing_prices[j+1]:
        change_closing = closing_prices[j+1] - closing_prices[j] 
        max_change_closing = max(change_closing, max_change_closing)
        
print("The largest change between any two days' closing prices: " + "$" + str(round(max_change_closing,2)) + ".")

The largest change between any two days' closing prices: $2.56.


### Task 6
##### What was the average daily trading volume during this year?

In [15]:
volumes = 0
volumes_count = 0
# get daily trading volumes
for i in response_2017[initial_key]['data']:
    if i[6] is not None:
        volumes += i[6]
        volumes_count += 1
avg_vol = (round(volumes/volumes_count,2))
print("The average daily trading volume in 2017: " + str(avg_vol) + ".")

The average daily trading volume in 2017: 89124.34.


### Task 7
##### (Optional) What was the median trading volume during this year?

This task, though optional, offers us a chance to implement a median function, which sounds interesting.

In [16]:
def median(initial_key = initial_key, key = 'data', index=6):
    res = []
    for i in response_2017[initial_key][key]:
        if i[index] is not None:
            res.append(i[index])
    res.sort()
    median_index = len(res)//2
    return res[median_index]

med = median()
print("The median daily trading volume in 2017: " + str(med) + ".")

The median daily trading volume in 2017: 76286.0.
